# Week3_1 Assignment

## [BASIC](#Basic) 
- 토크나이징이 완료된 위키 백과 코퍼스를 다운받고 **단어 사전을 구축하는 함수를 구현**할 수 있다.
- `Skip-Gram` 방식의 학습 데이터 셋을 생성하는 **Dataset과 Dataloader 클래스를 구현**할 수 있다.
- **Negative Sampling** 함수를 구현할 수 있다. 


## [CHALLENGE](#Challenge)
- Skip-Gram을 학습 과정 튜토리얼을 따라하며, **Skip-Gram을 학습하는 클래스를 구현**할 수 있다. 


## [ADVANCED](#Advanced)
- Skip-Gram 방식으로 word embedding을 학습하는 **Word2Vec 클래스를 구현**하고 실제로 학습할 수 있다.
- 학습이 완료된 word embedding을 불러와 **Gensim 패키지를 사용해 유사한 단어**를 뽑을 수 있다. 

### Reference
- [Skip-Gram negative sampling 한국어 튜토리얼](https://wikidocs.net/69141)
    - (참고) 위 튜토리얼에서는 target word와 context word 페어의 레이블은 1로, target word와 negative sample word 페어의 레이블은 0이 되도록 학습 데이터를 구현해 binary classification을 구현한다. 하지만 우리는 word2vec 논문 방식을 그대로 따르기 위해 label을 생성하지 않고 대신 loss 함수를 변행해서 binary classification을 학습할 것이다. 

In [ ]:
import os 
import sys
import pandas as pd
import numpy as np
import re
from typing import List, Dict
import random

In [ ]:
!pip install transformers

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [ ]:
# seed
seed = 7777
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [ ]:
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla P100-PCIE-16GB
cuda


## Basic

### 토크나이징이 완료된 위키 백과 코퍼스 다운로드 및 불용어 사전 크롤링
- 나의 구글 드라이브에 데이터를 다운받아 영구적으로 사용할 수 있도록 하자. 
    - [데이터 다운로드 출처](https://ratsgo.github.io/embedding/downloaddata.html)
- 다운받은 데이터는 토크나이징이 완료된 상태이지만 불용어를 포함하고 있다. 따라서 향후 불용어를 제거하기 위해 불용어 사전을 크롤링하자. 
    - [불용어 사전 출처](https://www.ranks.nl/stopwords/korean)

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd  /content/drive/MyDrive/[Wanted]week3-1

/content/drive/MyDrive/[Wanted]week3-1


In [ ]:
# 데이터 다운로드
# !pip install gdown
# !gdown https://drive.google.com/u/0/uc?id=1Ybp_DmzNEpsBrUKZ1-NoPDzCMO39f-fx


# 권한 문제로 링크로부터 별도로 다운받은 후 옮기기
# !unzip tokenized.zip

In [ ]:
# 한국어 불용어 리스트 크롤링
import requests
from bs4 import BeautifulSoup

url = "https://www.ranks.nl/stopwords/korean"
response = requests.get(url, verify = False)

if response.status_code == 200:
    soup = BeautifulSoup(response.text,'html.parser')
    content = soup.select_one('#article178ebefbfb1b165454ec9f168f545239 > div.panel-body > table > tbody > tr')
    stop_words=[]
    for x in content.strings:
        x=x.strip()
        if x:
            stop_words.append(x)
    print(f"# Korean stop words: {len(stop_words)}")
else:
    print(response.status_code)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


# Korean stop words: 677


In [ ]:
stop_words = set(stop_words)
stop_words

{'가',
 '가까스로',
 '가령',
 '각',
 '각각',
 '각자',
 '각종',
 '갖고말하자면',
 '같다',
 '같이',
 '개의치않고',
 '거니와',
 '거바',
 '거의',
 '것',
 '것과 같이',
 '것들',
 '게다가',
 '게우다',
 '겨우',
 '견지에서',
 '결과에 이르다',
 '결국',
 '결론을 낼 수 있다',
 '겸사겸사',
 '고려하면',
 '고로',
 '곧',
 '공동으로',
 '과',
 '과연',
 '관계가 있다',
 '관계없이',
 '관련이 있다',
 '관하여',
 '관한',
 '관해서는',
 '구',
 '구체적으로',
 '구토하다',
 '그',
 '그들',
 '그때',
 '그래',
 '그래도',
 '그래서',
 '그러나',
 '그러니',
 '그러니까',
 '그러면',
 '그러므로',
 '그러한즉',
 '그런 까닭에',
 '그런데',
 '그런즉',
 '그럼',
 '그럼에도 불구하고',
 '그렇게 함으로써',
 '그렇지',
 '그렇지 않다면',
 '그렇지 않으면',
 '그렇지만',
 '그렇지않으면',
 '그리고',
 '그리하여',
 '그만이다',
 '그에 따르는',
 '그위에',
 '그저',
 '그중에서',
 '그치지 않다',
 '근거로',
 '근거하여',
 '기대여',
 '기점으로',
 '기준으로',
 '기타',
 '까닭으로',
 '까악',
 '까지',
 '까지 미치다',
 '까지도',
 '꽈당',
 '끙끙',
 '끼익',
 '나',
 '나머지는',
 '남들',
 '남짓',
 '너',
 '너희',
 '너희들',
 '네',
 '넷',
 '년',
 '논하지 않다',
 '놀라다',
 '누가 알겠는가',
 '누구',
 '다른',
 '다른 방면으로',
 '다만',
 '다섯',
 '다소',
 '다수',
 '다시 말하자면',
 '다시말하면',
 '다음',
 '다음에',
 '다음으로',
 '단지',
 '답다',
 '당신',
 '당장',
 '대로 하다',
 '대하면',
 '대하여',
 '대해 말하자면',
 '대해서',
 '댕그',


### 단어 사전 구축 함수 구현 
- 문서 리스트를 입력 받아 사전을 생성하는 `make_vocab()` 함수를 구현하라.
- 함수 정의
    - 입력 매개변수
        - docs : 문서 리스트
        - min_count : 최소 단어 등장 빈도수 (단어 빈도가 `min_count` 미만인 단어는 사전에 포함하지 않음)
    - 조건
        - 문서 길이 제한
            - 단어 개수가 3개 이하인 문서는 처리하지 않음. (skip)
        - 사전에 포함되는 단어 빈도수 제한
            - 단어가 빈도가 `min_count` 미만은 단어는 사전에 포함하지 않음.
        - 불용어 제거 
            - 불용어 리스트에 포함된 단어는 제거 
    - 반환값 
        - word2count : 단어별 빈도 사전 (key: 단어, value: 등장 횟수)
        - wid2word : 단어별 인덱스(wid) 사전 (key: 단어 인덱스(int), value: 단어)
        - word2wid : 인덱스(wid)별 단어 사전 (key: 단어, value: 단어 인덱스(int))

In [ ]:
# 코퍼스 로드
with open("/content/drive/MyDrive/[Wanted]week3-1/tokenized/wiki_ko_mecab.txt",'r') as f:
  docs = f.read().split("\n") # delimiter : enter(\n)

In [ ]:
print(f"# wiki documents: {len(docs):,}")

# wiki documents: 311,238


In [ ]:
# 문서 개수를 500개로 줄임
docs=random.sample(docs,500)

In [ ]:
print(f"# wiki documents: {len(docs):,}")

# wiki documents: 500


In [ ]:
docs[11]

'남양 초등 학교 백금 분교 는 충청남도 청양군 남 양면 백 금리 2 7 3 - 2 에 있 었 던 공립 초등 학교 이 다 . * 1 9 6 4 . 9 . 1 . 설립 인가 ( 5 교실 ) * 1 9 6 4 . 1 0 . 1 . 개교 ( 8 학급 인가 ) * 1 9 7 0 . 3 . 1 . 1 2 학급 인가 * 1 9 7 7 . 3 . 1 . 1 1 학급 인가 * 1 9 8 6 . 1 . 1 . 벽지 학교 지정 * 1 9 8 6 . 3 . 1 . 6 학급 인가 * 1 9 8 7 . 1 . 5 . 시 , 군 , 구 , 읍 , 면 의 관할 구역 변경 에 따른 학교 명칭 및 위치 ( 주소 ) 변경 * 1 9 9 3 . 3 . 1 . 5 학급 인가 * 1 9 9 8 . 3 . 1 . 3 학급 인가 * 2 0 0 6 . 3 . 1 . 남양 초등 학교 백금 분교장 ( 2 학급 인가 ) * 2 0 0 6 . 9 . 1 . 남양 초등 학교 로 통합 백금 은 백 월산 의 기점 과 종점 으로 이용 되 는 마을 금곡 은 ‘ 거문고 골짜기 ’ 라는 뜻 으로 , 마을 지형 이 거문고 처럼 생겼 다 해서 붙여진 이름 으로서 마을 이름 을 따 서 백금 초등 학교 라 지 음 분류 : 남양 초등 학교 ( 충남 ) 분류 : 1 9 6 4 년 개교 분류 : 2 0 0 6 년 폐교 분류 : 대한민국 의 없 어 진 초등 학교 분류 : 청양군 의 학교 분류 : 초등 학교 분교'

In [ ]:
# 문서 내 숫자, 영어 대소문자, 특수문자를 제거 (re package 사용)
import re
def preprocess_line(line):
    line = re.sub(r'[^가-힣]+', r" ", line) # 숫자 제거
    
    # line = re.sub(r"[0-9]+", r" ", line) # 숫자 제거
    # line = re.sub(r"[a-zA-Z]+", r" ", line) # 알파벳 대소문자 제거
    # line = re.sub(r"[~!@#$%<>^&*()-=+_`\"?》《–・〈〉“”‘’·…■]+", r" ", line) # 특수문자 제거
    # line = re.sub(r"[一-龥]+", r" ", line) # 한자 제거

    # line = re.sub(r"[\*]+", r" ", line) # 기타 \로시작하는 알수 없는 토큰 제거
    
    line = re.sub(r'[" "]+', r" ",line) # 공백이 길경우 하나로 줄이기
    # line = line.strip()
    return line
preprocess_line(docs[11])

'남양 초등 학교 백금 분교 는 충청남도 청양군 남 양면 백 금리 에 있 었 던 공립 초등 학교 이 다 설립 인가 교실 개교 학급 인가 학급 인가 학급 인가 벽지 학교 지정 학급 인가 시 군 구 읍 면 의 관할 구역 변경 에 따른 학교 명칭 및 위치 주소 변경 학급 인가 학급 인가 남양 초등 학교 백금 분교장 학급 인가 남양 초등 학교 로 통합 백금 은 백 월산 의 기점 과 종점 으로 이용 되 는 마을 금곡 은 거문고 골짜기 라는 뜻 으로 마을 지형 이 거문고 처럼 생겼 다 해서 붙여진 이름 으로서 마을 이름 을 따 서 백금 초등 학교 라 지 음 분류 남양 초등 학교 충남 분류 년 개교 분류 년 폐교 분류 대한민국 의 없 어 진 초등 학교 분류 청양군 의 학교 분류 초등 학교 분교'

In [ ]:
docs = list(map(preprocess_line,docs))
# docs[11]

In [ ]:
print(f"Check : {docs[0][:1000]}")

Check : 남모 공주 는 신라 의 공주 왕족 으로 법흥왕 과 보과 공주 부여 씨 의 딸 이 며 백제 동성왕 의 외손녀 였 다 경쟁자 인 준정 과 함께 신라 의 초대 여성 원화 화랑 였 다 그 가 준정 에게 암살 당한 것 을 계기 로 화랑 은 여성 이 아닌 남성 미소년 으로 선발 하 게 되 었 다 신라 진흥왕 에게 는 사촌 누나 이 자 이모 가 된다 신라 의 청소년 조직 이 었 던 화랑도 는 처음 에 는 남모 준정 두 미녀 를 뽑 아 이 를 원화 라 했으며 이 들 주위 에 는 여 명 의 무리 를 따르 게 하 였 다 그러나 준정 과 남모 는 서로 최고 가 되 고자 시기 하 였 다 준정 은 박영실 을 섬겼 는데 지소태후 는 자신 의 두 번 째 남편 이 기 도 한 그 를 싫어해서 준정 의 원화 를 없애 고 낭도 가 부족 한 남모 에게 위화랑 의 낭도 를 더 해 주 었 다 그 뒤 남모 는 준정 의 초대 로 그 의 집 에 갔 다가 억지로 권하 는 술 을 받아마시 고 취한 뒤 준정 에 의해 강물 에 던져져 살해 되 었 다 이 일 이 발각 돼 준 정도 사형 에 처해지 고 나라 에서 는 귀족 출신 의 잘 생기 고 품행 이 곧 은 남자 를 뽑 아 곱 게 단장 한 후 이 를 화랑 이 라 칭하 고 받들 게 하 였 다 부왕 신라 제 대 국왕 법흥왕 모후 보과 공주 부여 씨 공주 남모 공주 외조부 백제 제 대 국왕 동성왕 외조모 신라 이찬 비지 의 딸 화랑전사 마루 년 배우 박효빈 신라 법흥왕 백제 동성왕 준정 화랑 분류 년 죽음 분류 신라 의 왕녀 분류 신라 의 왕족 분류 화랑 분류 암살 된 사람 분류 독살 된 사람 분류 법흥왕


In [ ]:
from collections import Counter

def make_vocab(docs:List[str], min_count:int):
    """
    'docs'문서 리스트를 입력 받아 단어 사전을 생성.
    
    return 
        - word2count : 단어별 빈도 사전
        - wid2word : 단어별 인덱스(wid) 사전 
        - word2wid : 인덱스(wid)별 단어 사전
    """

    word2count = dict()
    word2id = dict()
    id2word = dict()

    
    for doc in tqdm(docs):
        word_list = doc.split()
        # 1. 문서 길이 제한
        if len(word_list)<4:
          continue;
        # 2. 임시 딕셔너리(_word2count)에 단어별 등장 빈도 기록

        _word2count=dict(Counter(word_list))
        word2count={**word2count, **{key:val for key,val in _word2count.items() if val>=min_count}}
        # 3. 불용어 제거
        dickeys=word2count.keys()
        for word in stop_words:
          if word in set(dickeys):
            word2count.pop(word, None)
        # 4. 토큰 최소 빈도를 만족하는 토큰만 사전에 추가
        for word in word2count:
          if word not in set(word2id.keys()):
            if word2id!={}:
              word2id[word] = max(word2id.values()) +1 
            else:
              word2id[word] = 1
        # id2word
    id2word={val:key for key,val in word2id.items()}
    
    return word2count, word2id, id2word

In [ ]:
word2count, word2id, id2word = make_vocab(docs, min_count=5)

100%|██████████| 500/500 [01:21<00:00,  6.15it/s]


In [ ]:
word2count

{'남모': 6,
 '공주': 34,
 '는': 14,
 '신라': 8,
 '였': 5,
 '다': 15,
 '준정': 9,
 '화랑': 155,
 '고': 6,
 '분류': 17,
 '헝가리': 7,
 '펜싱': 7,
 '선수': 14,
 '하계': 6,
 '올림픽': 9,
 '레이다': 18,
 '실전': 5,
 '배치': 5,
 '은': 5,
 '목록': 13,
 '강': 5,
 '목': 27,
 '아과': 17,
 '붉': 10,
 '디': 5,
 '움': 5,
 '풀': 12,
 '엘라': 11,
 '라': 7,
 '수': 6,
 '있': 6,
 '파일': 6,
 '며': 18,
 '시동': 6,
 '실행': 8,
 '할': 9,
 '명령어': 8,
 '윈도': 10,
 '버전': 6,
 '포함': 9,
 '한': 6,
 '도스': 15,
 '컴퓨터': 10,
 '변수': 5,
 '사용': 5,
 '지': 5,
 '되': 12,
 '된다': 10,
 '키보드': 5,
 '설정': 10,
 '드라이버': 9,
 '줄': 5,
 '기본': 6,
 '적': 5,
 '구성': 6,
 '청안': 6,
 '학교': 5,
 '선남': 5,
 '초등': 6,
 '라마': 8,
 '장갑순양함': 6,
 '었': 10,
 '함': 9,
 '했': 5,
 '웹툰': 8,
 '연재': 9,
 '윤인완': 7,
 '게': 5,
 '멀티': 5,
 '버스': 11,
 '인류': 6,
 '대': 5,
 '앤': 40,
 '불린': 32,
 '헨리': 9,
 '세': 7,
 '엘리자베스': 5,
 '결혼': 6,
 '잉글랜드': 5,
 '후': 5,
 '받': 5,
 '던': 8,
 '왕비': 9,
 '캐서린': 8,
 '그녀': 13,
 '메리': 5,
 '프랑스': 8,
 '았': 5,
 '시녀': 5,
 '도': 7,
 '공작': 5,
 '사람': 5,
 '왕': 5,
 '참수': 5,
 '인가': 5,
 '학급': 9,
 '코지마': 6,
 '하루카': 7,
 '유이': 

In [ ]:
doc_len = sum(word2count.values()) # 문서 내 모든 단어의 개수 (단어별 등장 빈도의 총 합)
print(f"{doc_len:,}")

34,755


In [ ]:
print(f"# unique word : {len(word2id):,}")

# unique word : 2,982


### Dataset 클래스 구현
- Skip-Gram 방식의 학습 데이터 셋(`Tuple(target_word, context_word)`)을 생성하는 `CustomDataset` 클래스를 구현하라.
- 클래스 정의
    - 생성자(`__init__()` 함수) 입력 매개변수
        - docs: 문서 리스트
        - word2id: 단어별 인덱스(wid) 사전
        - window_size: Skip-Gram의 윈도우 사이즈
    - 메소드
        - `make_pair()`
            - 문서를 단어로 쪼개고, 사전에 존재하는 단어들만 단어 인덱스로 변경
            - Skip-gram 방식의 `(target_word, context_word)` 페어(tuple)들을 `pairs` 리스트에 담아 반환
        - `__len__()`
            - `pairs` 리스트의 개수 반환
        - `__getitem__(index)`
            - `pairs` 리스트를 인덱싱
    - 주의 사항
        - `nn.Module`를 부모 클래스로 상속 받음 


In [ ]:
class CustomDataset(Dataset):
    """
    문서 리스트를 받아 skip-gram 방식의 (target_word, context_word) 데이터 셋을 생성
    """
    def __init__(self, docs:List[str], word2id:Dict[str,int], window_size:int=5):
        self.docs = docs
        self.word2id = word2id
        if window_size%2==1:
          self.window_size = window_size
        else :
          print("window size is not proper. try again")
        assert window_size%2==1
        self.pairs = self.make_pair()
    
    def make_pair(self):
        """
        (target, context) 형식의 Skip-gram pair 데이터 셋 생성 
        """
        W = self.window_size
        pairs = []
        for doc in self.docs:
          doclist= doc.split()
          doclist=list(map(lambda x: self.word2id.get(x),doclist)) # id값 mapping
          doclist=[i for i in doclist if i] # remove None element 
          N=len(doclist)
          
          tups=[]
          for i in range(N-W+1): # i~i+4 
            nhalf = int((W//2)/2)
            # mid= i+nhalf
            # tup_front = [(doclist[i+j],doclist[mid]) for j in range(nhalf)]
            # tup_back = [(doclist[(mid+1)+j],doclist[mid]) for j in range(nhalf)]
            sublist= [(doclist[i+j],doclist[i+nhalf]) for j in range(W) if j!=nhalf]
            tups.extend(sublist)
          pairs.extend(tups)
        return pairs
        
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        return self.pairs[idx][0], self.pairs[idx][1]

In [ ]:
dataset = CustomDataset(docs, word2id, window_size=5)

In [ ]:
len(dataset)

531204

In [ ]:
dataset[0]

(1, 2)

In [ ]:
# verify (target word, context word)
for i, pair in enumerate(dataset):
    if i==100:
        break
    print(f"({id2word[pair[0]]}, {id2word[pair[1]]})")
    

(남모, 공주)
(는, 공주)
(신라, 공주)
(공주, 공주)
(공주, 는)
(신라, 는)
(공주, 는)
(공주, 는)
(는, 신라)
(공주, 신라)
(공주, 신라)
(부여, 신라)
(신라, 공주)
(공주, 공주)
(부여, 공주)
(씨, 공주)
(공주, 공주)
(부여, 공주)
(씨, 공주)
(딸, 공주)
(공주, 부여)
(씨, 부여)
(딸, 부여)
(며, 부여)
(부여, 씨)
(딸, 씨)
(며, 씨)
(였, 씨)
(씨, 딸)
(며, 딸)
(였, 딸)
(다, 딸)
(딸, 며)
(였, 며)
(다, 며)
(인, 며)
(며, 였)
(다, 였)
(인, 였)
(준정, 였)
(였, 다)
(인, 다)
(준정, 다)
(신라, 다)
(다, 인)
(준정, 인)
(신라, 인)
(초대, 인)
(인, 준정)
(신라, 준정)
(초대, 준정)
(여성, 준정)
(준정, 신라)
(초대, 신라)
(여성, 신라)
(화랑, 신라)
(신라, 초대)
(여성, 초대)
(화랑, 초대)
(였, 초대)
(초대, 여성)
(화랑, 여성)
(였, 여성)
(다, 여성)
(여성, 화랑)
(였, 화랑)
(다, 화랑)
(준정, 화랑)
(화랑, 였)
(다, 였)
(준정, 였)
(화랑, 였)
(였, 다)
(준정, 다)
(화랑, 다)
(은, 다)
(다, 준정)
(화랑, 준정)
(은, 준정)
(여성, 준정)
(준정, 화랑)
(은, 화랑)
(여성, 화랑)
(남성, 화랑)
(화랑, 은)
(여성, 은)
(남성, 은)
(게, 은)
(은, 여성)
(남성, 여성)
(게, 여성)
(되, 여성)
(여성, 남성)
(게, 남성)
(되, 남성)
(었, 남성)
(남성, 게)
(되, 게)
(었, 게)
(다, 게)


### 위에서 생성한 `dataset`으로 DataLoader  객체 생성
- `DataLoader` 클래스로 `train_dataloader`객체를 생성하라. 
    - 생성자 매개변수와 값
        - dataset = 위에서 생성한 dataset
        - batch_size = 64
        - shuffle = True

In [ ]:
train_dataloader = DataLoader(dataset=dataset, batch_size=64, shuffle=True)

In [ ]:
len(train_dataloader)

8301

### Negative Sampling 함수 구현
- Skip-Gram은 복잡도를 줄이기 위한 방법으로 negative sampling을 사용한다. 
- `sample_table`이 다음과 같이 주어졌을 때, sample_table에서 랜덤으로 값을 뽑아 (batch_size, n_neg_sample) shape의 matrix를 반환하는 `get_neg_v_negative_sampling()`함수를 구현하라. 
- Sample Table은 negative distribution을 따른다. 
    - [negative distribution 설명](https://aegis4048.github.io/optimize_computational_efficiency_of_skip-gram_with_negative_sampling#How-are-negative-samples-drawn?)
- 함수 정의
    - 입력 매개변수
        - batch_size : 배치 사이즈, matrix의 row 개수 
        - n_neg_sample : negative sample의 개수, matrix의 column 개수
    - 반환값 
        - neg_v : 추출된 negative sample (2차원의 리스트)


In [ ]:
# negative sample을 추출할 sample table 생성 (해당 코드를 참고)
sample_table = []
sample_table_size = doc_len

# noise distribution 생성
alpha = 3/4
frequency_list = np.array(list(word2count.values())) ** alpha
Z = sum(frequency_list)
ratio = frequency_list/Z
negative_sample_dist = np.round(ratio*sample_table_size)

for wid, c in enumerate(negative_sample_dist):
    sample_table.extend([wid]*int(c))

In [ ]:
len(sample_table) # word 개수 * sample table size

35179

In [ ]:
def get_neg_v_negative_sampling(batch_size:int, n_neg_sample:int):
    """
    위에서 정의한 sample_table에서 (batch_size, n_neg_sample) shape만큼 랜덤 추출해 "네거티브 샘플 메트릭스"를 생성
    np.random.choice() 함수 활용 (위에서 정의한 sample_table을 함수의 argument로 사용)
    """
    neg_v = np.random.choice(sample_table, size= (batch_size,n_neg_sample))
    
    return neg_v

In [ ]:
get_neg_v_negative_sampling(4, 5)

array([[ 918,  778, 1219,  723, 2216],
       [2771, 2270, 2364, 2736,  438],
       [2213, 1305,  571, 1773, 2497],
       [2176, 1305, 1516, 2765, 1398]])

## Challenge

### 미니 튜토리얼
- 아래 튜토리얼을 따라하며 Skip-Gram 모델의 `forward` 및 `loss` 연산 방식을 이해하자
- Reference
    - [torch.nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)
    - [torch bmm](https://pytorch.org/docs/stable/generated/torch.bmm.html)
    - [Skip-Gram negative sampling loss function 설명 영문 블로그](https://aegis4048.github.io/optimize_computational_efficiency_of_skip-gram_with_negative_sampling#Derivation-of-Cost-Function-in-Negative-Sampling)
    - [Skip-Gram negative sampling loss function 설명 한글 블로그](https://reniew.github.io/22/)

In [ ]:
# hyper parameter example
emb_size = 30000 # vocab size
emb_dimension = 300 # word embedding 차원
n_neg_sample = 5
batch_size = 32

In [ ]:
# 1. Embedding Matrix와 Context Matrix를 생성
u_embedding = nn.Embedding(emb_size, emb_dimension, sparse=True).to(device)
v_embedding = nn.Embedding(emb_size, emb_dimension, sparse=True).to(device)

In [ ]:
# 2. wid(단어 인덱스)를 임의로 생성
pos_u = torch.randint(high = emb_size, size = (batch_size,))
pos_v = torch.randint(high = emb_size, size = (batch_size,))
neg_v = get_neg_v_negative_sampling(batch_size, n_neg_sample)
print(f"Target word idx : {pos_u} Pos context word idx : {pos_v} Neg context word idx : {neg_v}\n")

Target word idx : tensor([24460, 10634,  2864, 23952,  3320, 15187, 19625, 26546, 27339,  3920,
        25847,  6023,  5055,  7070,  6291, 10245, 15926,   641, 20178,  4565,
         4784, 26715, 16955, 28742, 17947, 19774,  8065, 22605,  3061, 28965,
         3056, 17963]) Pos context word idx : tensor([23224,  5636, 23712,  5234,  3991, 17897, 25123, 17938, 19634, 24228,
          693,   799, 25457,  1308, 28935, 25696,  5601, 23878,  8312,  1292,
        21380, 16974,  9318,  9578, 12915, 29271, 26465, 20572,  2362, 25929,
        19754, 29080]) Neg context word idx : [[1351 2581 1622 1621 1395]
 [2314  488 1018 1244 1222]
 [1680 2146  503 1858  122]
 [  76 1128 1293  498 1227]
 [1949  639  486  973 1288]
 [ 547 2608 2482  334 1187]
 [1527 1646 1622 2677 2601]
 [1768 2909 2160 1749 1406]
 [ 971 1342 1251 2242 1672]
 [2201 2524 1959 2707  584]
 [1699 1595  472  285 1212]
 [1375 1945 1690  674 2436]
 [1107  153  792 2852 2543]
 [1668 1347  836  898 2471]
 [2186 2119 1749 1627 1513]
 [

In [ ]:
# 3. tensor로 변환
pos_u = Variable(torch.LongTensor(pos_u)).to(device)
pos_v = Variable(torch.LongTensor(pos_v)).to(device)
neg_v = Variable(torch.LongTensor(neg_v)).to(device)

In [ ]:
# 4. wid로 각각의 embedding matrix에서 word embedding 값을 가져오기
pos_u = u_embedding(pos_u)
pos_v = v_embedding(pos_v)
neg_v = v_embedding(neg_v)
print(f"shape of pos_u embedding : {pos_u.shape}\n shape of pos_v embedding : {pos_v.shape}\n shape of neg_v embedding : {neg_v.shape}")


shape of pos_u embedding : torch.Size([32, 300])
 shape of pos_v embedding : torch.Size([32, 300])
 shape of neg_v embedding : torch.Size([32, 5, 300])


In [ ]:
# 5. dot product 
pos_score = torch.mul(pos_u, pos_v) # 행렬 element-wise 곱
pos_score = torch.sum(pos_score, dim=1)
print(f"shape of pos logits : {pos_score.shape}\n")

# input : (b,5,300) , (b,300, -1) -> (b,5) (squeezed) 
neg_score = torch.bmm(neg_v, pos_u.unsqueeze(dim=2)).squeeze() # batch 별 matrix 연산
print(f"shape of logits : {neg_score.shape}")

shape of pos logits : torch.Size([32])

shape of logits : torch.Size([32, 5])


In [ ]:
# 6. loss 구하기
pos_score = F.logsigmoid(pos_score)
print(pos_score)
neg_score = F.logsigmoid(-1*neg_score) # negative의 logit은 minimize 하기 위해 -1 곱함
print(neg_score)
print(f"pos logits : {pos_score.sum()}")
print(f"neg logits : {neg_score.sum()}")
loss = -1 * (torch.sum(pos_score) + torch.sum(neg_score))
print(f"Loss : {loss}")

tensor([-7.4414e+00, -1.5843e-01, -5.9454e+00, -2.3842e-07, -7.5337e-05,
        -1.1106e+01, -0.0000e+00, -2.7479e+01, -1.8868e+01, -9.7658e-02,
        -1.4674e+01, -0.0000e+00, -1.0167e+01, -1.5579e-04, -1.3113e-06,
        -0.0000e+00, -1.3921e+01, -1.9878e-02, -7.5666e+00, -3.9380e+01,
        -7.6009e+00, -2.7195e+00, -3.5763e-07, -5.8717e-04, -2.3842e-07,
        -6.5446e+00, -1.1921e-07, -3.1243e+01, -1.1446e+00, -2.3842e-07,
        -9.0385e+00, -2.6304e+01], device='cuda:0',
       grad_fn=<LogSigmoidBackward0>)
tensor([[-9.5824e+00, -4.5648e-02, -1.3385e-02, -0.0000e+00, -2.9583e-04],
        [-2.0766e-03, -8.4657e+00, -1.7236e-04, -9.8981e+00, -8.0168e+00],
        [-2.0449e+01, -4.2255e-01, -8.7510e-03, -8.3311e-03, -1.1063e+01],
        [-6.1989e-06, -1.8432e+01, -1.9700e+01, -0.0000e+00, -5.4819e+00],
        [-1.1330e+01, -3.9349e-03, -1.9016e+01, -2.3590e-03, -0.0000e+00],
        [-1.0307e-02, -0.0000e+00, -1.7200e-01, -1.2480e-04, -7.9647e+00],
        [-3.5763e-07, 

### Skip-gram 클래스 구현
- Skip-Gram 방식으로 단어 embedding을 학습하는 `SkipGram` 클래스를 구현하라.
- 클래스 정의
    - 생성자(`__init__()` 함수) 입력 매개변수
        - `vocab_size` : 사전내 단어 개수
        - `emb_dimension` : 엠베딩 크기
        - `device` : 연산 장치 종류
    - 생성자에서 생성해야할 변수 
        - `vocab_size` : 사전내 단어 개수
        - `emb_dimension` : 엠베딩 크기
        - `u_embedding` : (vocab_size, emb_dimension) 엠베딩 메트릭스 (target_word)
        - `v_embedding` : (vocab_size, emb_dimension) 엠베딩 메트릭스 (context_word)
    - 메소드
        - `init_embedding()` (제공됨)
            - 엠베딩 메트릭스 값을 초기화
        - `forward()`
            - 위 튜토리얼과 같이 dot product를 수행한 후 score를 생성
            - loss를 반환 (loss 설명 추가)
        - `save_emedding()` (제공됨)
            - `u_embedding`의 단어 엠베딩 값을 단어 별로 파일에 저장
    - 주의 사항     
        - `nn.Module`를 부모 클래스로 상속 받음 

In [ ]:
class SkipGram(nn.Module):
    def __init__(self, vocab_size:int, emb_dimension:int, device:str):
        super(SkipGram, self).__init__()
        self.vocab_size = vocab_size
        self.emb_dimension = emb_dimension
        self.u_embedding = nn.Embedding(vocab_size, emb_dimension, sparse=True).to(device)
        self.v_embedding = nn.Embedding(vocab_size, emb_dimension, sparse=True).to(device)
        self.init_embedding()
    
    
    def init_embedding(self):
        """
        u_embedding과 v_embedding 메트릭스 값을 초기화
        """
        initrange = 0.5 / self.emb_dimension
        self.u_embedding.weight.data.uniform_(-initrange, initrange)
        self.v_embedding.weight.data.uniform_(-0, 0) # u, v matrix를 initialize
    
    
    def forward(self, pos_u, pos_v, neg_v):
        """
        dot product를 수행한 후 score를 생성
        loss 반환
        """    
            
        # 각각의 embedding matrix에서 word embedding 값을 가져오기
        pos_u = self.u_embedding(pos_u)
        pos_v = self.v_embedding(pos_v)
        neg_v = self.v_embedding(neg_v)
        
        # dot product 
        pos_score = torch.sum(torch.mul(pos_u, pos_v),dim=1) #torch.mul 사용 후 vocab 별로 합 구하기
        # input : (b,n_neg_sample,dimension) , (b,dimension, -1) -> (b,5) (squeezed) 
        neg_score = torch.bmm(neg_v, pos_u.unsqueeze(dim=2)).squeeze() # batch 별 matrix 연산
        
        # loss 구하기
        pos_score = F.logsigmoid(pos_score)
        neg_score = F.logsigmoid(-1*neg_score) # negative의 logit은 minimize 하기 위해 -1 곱함

        loss = -1 * (torch.sum(pos_score) + torch.sum(neg_score))

        return loss
    
    def save_embedding(self, id2word, file_name, use_cuda):
        """
        'file_name' 위치에 word와 word_embedding을 line-by로 저장
        파일의 첫 줄은 '단어 개수' 그리고 '단어 embedding 사이즈' 값을 입력해야 함
        """
        if use_cuda: # parameter를 gpu 메모리에서 cpu 메모리로 옮김
            embedding = self.u_embedding.weight.cpu().data.numpy()
        else:
            embedding = self.u_embedding.weight.data.numpy()

        with open(file_name, 'w') as writer:
            # 파일의 첫 줄은 '단어 개수' 그리고 '단어 embedding 사이즈' 값을 입력해야 함
            writer.write(f"{len(id2word)} {embedding.shape[-1]}\n")
            
            for wid, word in id2word.items():
                e = embedding[wid]
                e = " ".join([str(e_) for e_ in e])
                writer.write(f"{word} {e}\n")

## Advanced

### Skip-Gram 방식의  Word2Vec 클래스 구현
- Skip-Gram 방식으로 단어 embedding을 학습하는 `Word2Vec` 클래스를 구현하라.
- 클래스 정의
    - 생성자(`__init__()`) 입력 매개 변수
        - `input_file` : 학습할 문서 리스트
        - `output_file_name` : 학습된 word embedding을 저장할 파일 위치
        - `device` : 연상 장치 종류
        - `emb_dimension` : word embedding 차원
        - `batch_size` : 학습 배치 사이즈
        - `window_size` : skip-gram 윈도우 사이즈 (context word 개수를 결정)
        - `n_neg_sample` : negative sample 개수
        - `iteration` : 학습 반복 횟수
        - `lr` : learning rate
        - `min_count` : 사전에 추가될 단어의 최소 등장 빈도
    - 생성자에서 생성해야 할 변수 
        - `docs` : 학습할 문서 리스트
        - `output_file_name` : 학습된 word embedding을 저장할 파일 위치
        - `word2count`, `word2id`, `id2word` : 위에서 구현한 `make_vocab()` 함수의 반환 값
        - `device` : 연산 장치 종류
        - `emb_size` : vocab의 (unique한) 단어 종류 
        - `emb_dimension` : word embedding 차원
        - `batch_size` : 학습 배치 사이즈
        - `window_size` : skip-gram 윈도우 사이즈 (context word 개수를 결정)
        - `n_neg_sample` : negative sample 개수
        - `iteration` : 학습 반복 횟수
        - `lr` : learning rate
        - `model` : `SkipGram` 클래스의 인스턴스
        - `optimizer` : `SGD` 클래스의 인스턴스
    - 메소드
        - `train()`
            - 입력 매개변수 
                - `train_dataloader`
            - Iteration 횟수만큼 input_file 학습 데이터를 학습한다. 매 epoch마다 for loop 돌면서 batch 단위 학습 데이터를 skip gram 모델에 학습함. 학습이 끝나면 word embedding을 output_file_name 파일에 저장.
- Reference
    - [Optimizer - SGD](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html)

In [ ]:
from torch.nn.modules.loss import L1Loss
from torch.nn import CrossEntropyLoss
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

class Word2Vec:
    def __init__(self, 
                input_file: List[str],
                output_file_name: str,
                 device: str,
                 emb_dimension=300,
                 batch_size = 64,
                 window_size=5,
                 n_neg_sample = 5,
                 iteration=1,
                 lr = 0.02,
                 min_count=5):
        self.docs = input_file
        self.output_file_name = '/content/output/'
        self.word2count, self.word2id, self.id2word = make_vocab(self.docs, min_count)
        self.device = device
        self.emb_size = len(self.word2id)+1
        self.emb_dimension = emb_dimension
        self.batch_size = batch_size
        self.window_size = window_size
        self.n_neg_sample = n_neg_sample
        self.iteration = iteration
        self.lr = lr
        # (self, vocab_size:int, emb_dimension:int, device:str)
        self.model = SkipGram(self.emb_size, self.emb_dimension, self.device)
        self.optimizer = torch.optim.SGD(self.model.parameters(),lr=self.lr) # torch.optim.SGD 클래스 사용
        # self.loss_fn=L1Loss()

        # train() 함수에서 만든 임베딩 결과 파일들을 저장할 폴더 생성 (os.makedirs 사용)
        if not os.path.exists('/content/output'):
          os.makedirs('/content/output')
        
    
    def train(self, train_dataloader):
        
        # lr 값을 조절하는 스케줄러 인스턴스 변수를 생성
        self.scheduler = get_linear_schedule_with_warmup(
            optimizer = self.optimizer,
            num_warmup_steps=0,
            num_training_steps=self.iteration*len(train_dataloader)
        )
        
        for epoch in range(self.iteration):
            
            print(f"*****Epoch {epoch} Train Start*****")
            print(f"*****Epoch {epoch} Total Step {len(train_dataloader)}*****")
            total_loss, batch_loss, batch_step = 0,0,0

            for step, batch in enumerate(train_dataloader):
                batch_step+=1

                pos_u, pos_v = batch
                # negative data 생성
                neg_v = get_neg_v_negative_sampling(pos_u.shape[0], self.n_neg_sample)
                
                # 데이터를 tensor화 & device 설정
                pos_u = Variable(torch.LongTensor(pos_u)).to(device)
                pos_v = Variable(torch.LongTensor(pos_v)).to(device)
                neg_v = Variable(torch.LongTensor(neg_v)).to(device)

                # model의 gradient 초기화
                self.model.zero_grad()
                # optimizer의 gradient 초기화
                self.optimizer.zero_grad()

                # forward
                loss = self.model.forward(pos_u, pos_v, neg_v)
                
                

                

                batch_loss += loss.item()
                total_loss += loss.item()
                # loss 계산
                loss.backward()
                # print(loss.item())
            
                # optimizer 업데이트
                self.optimizer.step()
                # scheduler 업데이트
                self.scheduler.step()

                
                
                
                if (step%500 == 0) and (step!=0):
                    print(f"Step: {step} Loss: {batch_loss/batch_step:.4f} lr: {self.optimizer.param_groups[0]['lr']:.4f}")
                    # 변수 초기화    
                    batch_loss, batch_step = 0,0
            
            print(f"Epoch {epoch} Total Mean Loss : {total_loss/(step+1):.4f}")
            print(f"*****Epoch {epoch} Train Finished*****\n")
            
            print(f"*****Epoch {epoch} Saving Embedding...*****")
            self.model.save_embedding(self.id2word, os.path.join(self.output_file_name, f'w2v_{epoch}.txt'), True if 'cuda' in self.device.type else False)
            print(f"*****Epoch {epoch} Embedding Saved at {os.path.join(self.output_file_name, f'w2v_{epoch}.txt')}*****\n")

In [ ]:
output_file = os.path.join(".", "word2vec_wiki")
# Word2Vec 클래스의 인스턴스 생성
w2v = Word2Vec(docs, output_file, device, n_neg_sample=10, iteration=3)

100%|██████████| 500/500 [01:25<00:00,  5.86it/s]


In [ ]:
# 학습 데이터 셋 및 데이터 로더 생성 (위에서 생성한 w2v의 attribute들을 argument에 적절히 넣기)

# CustomDataset(docs, word2id, window_size=5)
dataset = CustomDataset(w2v.docs, w2v.word2id, w2v.window_size)

# train_dataloader = DataLoader(dataset=dataset, batch_size, shuffle=True)
train_dataloader = DataLoader(dataset=dataset, batch_size=w2v.batch_size, shuffle=True)
len(train_dataloader)

8301

In [ ]:
# 학습

w2v.train(train_dataloader)

*****Epoch 0 Train Start*****
*****Epoch 0 Total Step 8301*****
Step: 500 Loss: 486.7337 lr: 0.0196
Step: 1000 Loss: 382.7275 lr: 0.0192
Step: 1500 Loss: 264.1092 lr: 0.0188
Step: 2000 Loss: 211.8946 lr: 0.0184
Step: 2500 Loss: 184.8337 lr: 0.0180
Step: 3000 Loss: 171.0326 lr: 0.0176
Step: 3500 Loss: 164.2032 lr: 0.0172
Step: 4000 Loss: 159.1978 lr: 0.0168
Step: 4500 Loss: 155.2258 lr: 0.0164
Step: 5000 Loss: 153.7880 lr: 0.0160
Step: 5500 Loss: 151.7665 lr: 0.0156
Step: 6000 Loss: 150.1942 lr: 0.0152
Step: 6500 Loss: 149.1978 lr: 0.0148
Step: 7000 Loss: 148.1623 lr: 0.0144
Step: 7500 Loss: 146.4712 lr: 0.0140
Step: 8000 Loss: 146.5109 lr: 0.0136
Epoch 0 Total Mean Loss : 199.6032
*****Epoch 0 Train Finished*****

*****Epoch 0 Saving Embedding...*****
*****Epoch 0 Embedding Saved at /content/output/w2v_0.txt*****

*****Epoch 1 Train Start*****
*****Epoch 1 Total Step 8301*****
Step: 500 Loss: 145.0469 lr: 0.0129
Step: 1000 Loss: 143.7887 lr: 0.0125
Step: 1500 Loss: 143.5078 lr: 0.0121


### 유사한 단어 확인
- 사전에 존재하는 단어들과 유사한 단어를 검색해보자. Gensim 패키지는 유사 단어 외에도 단어간의 유사도를 계산하는 여러 함수를 제공한다. 실험을 통해 word2vec의 한계점을 발견했다면 아래에 markdown으로 작성해보자. 
- [Gensim 패키지 document](https://radimrehurek.com/gensim/models/keyedvectors.html)

In [ ]:
import gensim

In [ ]:
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('/content/output/w2v_0.txt', binary=False)

In [ ]:
word_vectors.most_similar(positive='삼겹살')

[('구체', 0.9998613595962524),
 ('동물', 0.9998601675033569),
 ('집단', 0.9998598694801331),
 ('다루', 0.9998542666435242),
 ('국민주의', 0.9998542070388794),
 ('실험', 0.9998539686203003),
 ('바', 0.999853253364563),
 ('우주', 0.9998530745506287),
 ('교황', 0.99985271692276),
 ('차지', 0.9998518824577332)]

word2vec의 한계점은?
- 실제로 매우 유사한 단어로 출력되는 것들이 정성적으로 납득이 잘 되지 않았다. 한 마디로, Negative sampling을 하고 skip-gram방식으로 학습하는 것이 생각보다 효과적이지는 않은 것 같다. training dataset이 작아서 그런 것도 있겠지만, 한마디로 성능이 좋지 않다. 
- 형태소(morpheme) 또는 subword 별로 의미론을 판단하기가 힘든 것들이 있다. 위의 결과 중 '국민' 과 '주의' 같은 경우 각기 다른 의미의 단어로 분해가 가능하다. Fasttext 모델로 개선이 가능한 것으로 알고 있는데 별도로 실험을 해보면 좋을 것 같다.